<a href="https://colab.research.google.com/github/dmdiegoar/Quant-code-t0/blob/main/XGB_Sentido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LA MEJORA DE GK

mas listo que ayer

In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from google.colab import files  # Para descargar el CSV en Colab
from scipy import stats


# Funciones de features
def add_lagged_price_features(df):
    for lag in range(1, 6):
        df[f'close_lag_{lag}'] = df['Close'].shift(lag)
    return df

def add_lagged_price_features(df, etiqueta="close_lag", dato="Close"):
    for lag in range(1, 6):
        df[f'{etiqueta}_{lag}'] = df[dato].shift(lag)
    return df


def calculate_RSI(series, period=7):
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_ROC(series, period=5):
    return ((series - series.shift(period)) / series.shift(period)) * 100

def calculate_PPO(series, fast_period=5, slow_period=9, signal_period=5):
    ema_fast = series.ewm(span=fast_period, adjust=False).mean()
    ema_slow = series.ewm(span=slow_period, adjust=False).mean()
    ppo = (ema_fast - ema_slow) / ema_slow * 100
    signal_line = ppo.ewm(span=signal_period, adjust=False).mean()
    histogram = ppo - signal_line
    return ppo, signal_line, histogram

def calculate_EWO(series, fast_period=5, slow_period=35, signal_period=5):
    ema_fast = series.ewm(span=fast_period, adjust=False).mean()
    ema_slow = series.ewm(span=slow_period, adjust=False).mean()
    ewo = (ema_fast - ema_slow) / ema_slow * 100
    signal_line = ewo.ewm(span=signal_period, adjust=False).mean()
    histogram = ewo - signal_line
    return ewo, signal_line, histogram

def calculate_volatility(series, window=20):
    return series.rolling(window).std().round(6)

def calculate_sma5(series, period=5):
    return series.rolling(window=period).mean().round(4)

def calculate_sma13(series, period=13):
    return series.rolling(window=period).mean().round(4)

def calculate_sma26(series, period=26):
    return series.rolling(window=period).mean().round(4)

def calculate_sma50(series, period=50):
    return series.rolling(window=period).mean().round(4)

def calculate_sma200(series, period=200):
    return series.rolling(window=period).mean().round(4)


#def calculate_earnings_season(df):
#    df['Is_Earnings_Season'] = df.index.month.isin([1, 4, 7, 10])
#    return df

#def calculate_christmas_rally(df):
#    df['Is_Christmas_Rally'] = df.index.month.isin([11, 12])
#    return df

def create_features(df):
    df = add_lagged_price_features(df, "close_lag", "Close")
    df = add_lagged_price_features(df, "open_lag", "Open")
    df = add_lagged_price_features(df, "high_lag", "High")
    df['Pct_change'] = df['Close'].pct_change()
    for lag in range(1, 6):
        df[f'lag_change{lag}'] = df['Pct_change'].shift(lag)
    df['RSI'] = calculate_RSI(df['Close'])
    df['ROC'] = calculate_ROC(df['Close'])
    df['PPO'], df['PPO_Signal'], df['PPO_Histogram'] = calculate_PPO(df['Close'])
    df['EWO'], df['EWO_Signal'], df['EWO_Histogram'] = calculate_EWO(df['Close'])
    df['SMA5'] = calculate_sma5(df['Close'])
    df['SMA13'] = calculate_sma13(df['Close'])
    df['SMA26'] = calculate_sma26(df['Close'])
    df['SMA50'] = calculate_sma50(df['Close'])
    df['SMA200'] = calculate_sma200(df['Close'])
    df['Volatility'] = calculate_volatility(df['Close'])
    df['Label'] = (df['Pct_change'] > 0).astype(int)
    df['Return'] = np.log(df['Close'] / df['Close'].shift())
    #df = calculate_earnings_season(df)
    #df = calculate_christmas_rally(df)
    df.dropna(inplace=True)
    return df

# Definir fecha de corte manualmente (cambiar diariamente)
end_date = dt.datetime(2025, 7, 23)  # Ejemplo: cambiar a 2025-07-18 mañana

tk =[ "ALUA.BA", "BBAR.BA", "BMA.BA", "COME.BA", "CRES.BA", "EDN.BA", "GGAL.BA", "IRSA.BA", "LOMA.BA", "METR.BA", "PAMP.BA", "SUPV.BA", "TECO2.BA", "TGNO4.BA", "TGSU2.BA", "TRAN.BA", "TXAR.BA", "VALO.BA", "YPFD.BA"]

# pinchas = "BYMA.BA" start_date dt.datetime(2021, 1, 1),  "CEPU.BA" start_date  dt.datetime(2018, 1, 1)

results = [] # el del test, para que no lo reinicie
resultsp = [] # las predicciones, para que no lo reinicie

for papel in tk:

  symbol=papel
  #symbol="COME.BA"
  # Fechas dinámicas
  start_date = dt.datetime(2001, 1, 1)  # Inicio fijo
  train_end = end_date - pd.Timedelta(days=780)  # 6 meses antes de end_date (ajustable)
  next_day = end_date + pd.Timedelta(days=1)  # Predicción para el día siguiente
  backtest_start = end_date - pd.Timedelta(days=2)  # Inicio del backtesting 6 meses antes de end_date (ajustable)

  # Descargar datos
  df = yf.download(symbol, start=start_date, end=end_date, auto_adjust=False)

  # Verificar datos

  for intentos in range(1, 20):
    if df.empty:
      print("No se pudieron descargar datos. Verifica el símbolo, las fechas o la conexión.")
      print(f"\n Reintentando {symbol}: {intentos} de 20")
      df = yf.download(symbol, start=start_date, end=end_date, auto_adjust=False)
      intentos +=1
      time.sleep(8)
    else:
      print(f"\n Datos descargados en {intentos} vueltas: {symbol} , seguimos")


  # Aplanar MultiIndex si existe
  if isinstance(df.columns, pd.MultiIndex):
      print("MultiIndex detectado en columnas. Aplanando...")
      df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
      print("Columnas asignadas después de aplanamiento:", df.columns.tolist())
      print("Últimas filas antes de corrección:", df.tail())

      # Corregir el orden de las columnas según tu mapeo
      columns = df.columns.tolist()
      df = df[[columns[4], columns[2], columns[3], columns[0], columns[5], columns[1]]]  # Reordenar
      df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']  # Asignar nombres correctos
      print("Últimas filas después de corrección:", df.tail())

  if isinstance(df.index, pd.MultiIndex):
      print("MultiIndex detectado en índice. Seleccionando ticker...")
      df = df.xs(symbol, level='Ticker', axis=0)
  df.index = pd.to_datetime(df.index)  # Asegurar índice datetime
  if not df.index.is_unique:
      print("Advertencia: Índice con fechas duplicadas. Eliminando duplicados...")
      df = df[~df.index.duplicated(keep='first')]

  # Verificar columnas
  print("Columnas del DataFrame después de descargar y corregir:")
  print(df.columns)

  df['Open']= df['Open'].round(2)
  df['High']= df['High'].round(2)
  df['Low']= df['Low'].round(2)
  df['Close']= df['Close'].round(2)
  df['Adj Close']= df['Adj Close'].round(2)

  print("Últimas filas del DataFrame antes de crear features:")
  print(df.tail())

  # Crear features
  df = create_features(df)

  # Verificar datos después de crear features
  print("\nÚltimas filas del DataFrame después de crear features:")
  print(df.tail())
  # Verificar datos después de crear features
  print("\nÚltimas filas del DataFrame después de crear features:")
  print(df.tail())
  print(df.columns)

  print("Distribucion de etiquetas")
  print(df["Label"].value_counts(normalize=True))
  correlation = df[features + ["Label"]].corr()["Label"].sort_values(ascending=False)
  print("Correlacion con label")
  print(correlation)

  # Seleccionar features
  features = ['RSI', 'ROC', 'PPO', 'PPO_Signal', 'PPO_Histogram', 'EWO', 'EWO_Signal', 'EWO_Histogram', 'Volatility', 'SMA5', 'SMA13', 'SMA26', 'SMA50', 'SMA200' ] + [f'lag_change{i}' for i in range(1, 6)] + \
            [f'close_lag_{i}' for i in range(1, 6)] + [f'open_lag_{i}' for i in range(1, 6)]+ [f'high_lag_{i}' for i in range(1, 6)]

  #features = ['RSI', 'ROC', 'PPO', 'PPO_Signal', 'PPO_Histogram', 'EWO', 'EWO_Signal', 'EWO_Histogram', 'Volatility', 'SMA5', 'SMA13', 'SMA26', 'SMA50', 'SMA200' ] + [f'lag_change{i}' for i in range(1, 6)] + \
  #          [f'close_lag_{i}' for i in range(1, 6)] + [f'open_lag_{i}' for i in range(1, 6)]+ [f'high_lag_{i}' for i in range(1, 6)]
  X = df[features]
  y = df['Label']

  # Dividir datos en entrenamiento y prueba
  X_train_full = X[df.index <= train_end]
  y_train_full = y[df.index <= train_end]
  X_test = X[(df.index > train_end) & (df.index <= end_date)]  # Hasta end_date
  y_test = y[(df.index > train_end) & (df.index <= end_date)]

  # Optimizar hiperparámetros con RandomizedSearchCV
  print("Optimizar hiperparámetros con RandomizedSearchCV")
  param_dist = {
      'learning_rate': [0.01, 0.05, 0.1, 0.2],
      'max_depth': [3, 5, 7, 9],
      'n_estimators': [100, 500, 900],
      'subsample': [0.6, 0.8, 1.0],
      'colsample_bytree': [0.6, 0.8, 1.0],
      'gamma': [0, 0.1, 0.2]
  }
  xgb = XGBClassifier(objective='binary:logistic', random_state=42)
  random_search = RandomizedSearchCV(xgb, param_distributions=param_dist, n_iter=20, cv=5, scoring='f1', n_jobs=-1, random_state=42)
  random_search.fit(X_train_full, y_train_full)
  print("Mejores hiperparámetros:", random_search.best_params_)

  # Usar el mejor modelo
  best_model = random_search.best_estimator_

  # Optimizar el umbral
  y_train_prob = best_model.predict_proba(X_train_full)[:, 1]
  thresholds = np.arange(0.28, 0.88, 0.01) # de 0.28 a 0.88  con pasos de 0.01????
  best_threshold = 0.5
  best_f1 = 0
  for threshold in thresholds:
      y_pred_threshold = (y_train_prob >= threshold).astype(int)
      f1 = f1_score(y_train_full, y_pred_threshold)
      if f1 > best_f1:
          best_f1 = f1
          best_threshold = threshold
  best_threshold.round(2)
  print("Mejor umbral:", best_threshold)



  # Backtesting con escalado dinámico para evitar data leak
  dates = df[(df.index >= backtest_start) & (df.index <= end_date)].index
  #results = []

  for test_date in dates:
      train_data = df[df.index < test_date].copy()
      if train_data.empty or train_data['Label'].isna().all():
          continue
      train_data = train_data.dropna()

      X_train_loop = train_data[features]
      y_train_loop = train_data['Label']

      scaler = StandardScaler()  # Reinicio dinámico del escalador
      X_train_scaled = scaler.fit_transform(X_train_loop)
      print(f"Escalando datos hasta {train_data.index[-1]} para predecir {test_date}")

      best_model.fit(X_train_scaled, y_train_loop)

      if test_date in df.index:
          test_row = df.loc[[test_date]][features]
          if test_row.empty:
              continue
          test_row_scaled = scaler.transform(test_row)

          prediction_prob = best_model.predict_proba(test_row_scaled)[0][1].round(4)
          prediction = 1 if prediction_prob >= best_threshold else 0

          real_direction = df.loc[test_date, 'Label'] if pd.notna(df.loc[test_date, 'Label']) else None
          close_price = df.loc[test_date, 'Close']

          is_correct = int(prediction == real_direction) if real_direction is not None else None
          data_date = df.index[df.index < test_date][-1] if not df[df.index < test_date].empty else None

          results.append({
              'Papel': papel,
              'Fecha Predicción': test_date,
              'Fecha Datos': data_date,
              'Predicción': 'Alcista' if prediction == 1 else 'Bajista',
              'Resultado Real': 'Alcista' if real_direction == 1 else 'Bajista' if real_direction == 0 else None,
              'Precio Cierre': close_price,
              'Probabilidad Alcista': prediction_prob,
              'Correcta': 'Sí' if is_correct == 1 else 'No' if is_correct == 0 else None
          })

  # Crear tabla de resultados
  results_df = pd.DataFrame(results)
  results_df.set_index('Fecha Predicción', inplace=True)

  # Mostrar resultados
  pd.set_option('display.max_columns', None)
  print("\nResultados del backtesting (hasta", end_date.strftime('%Y-%m-%d'), "):")
  print("Nota: 'Fecha Predicción' es la fecha predicha; 'Fecha Datos' es la fecha de los datos usados.")
  print(results_df)

  # Guardar y descargar el CSV
  #results_df.to_csv(f"backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv", sep=";")
  #files.download(f"backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv")
  #print(f"\nArchivo 'backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv' generado y descargado.")

  # Métricas del backtesting
  #if results_df['Correcta'].notna().sum() > 0:
  #    accuracy = (results_df['Correcta'] == 'Sí').sum() / results_df['Correcta'].notna().sum()
  #    print(f"\nAccuracy del backtesting: {accuracy:.2%}")

  #    valid_results = results_df[results_df['Correcta'].notna()]
  #    y_true = [1 if r == 'Alcista' else 0 for r in valid_results['Resultado Real']]
  #    y_pred = [1 if p == 'Alcista' else 0 for p in valid_results['Predicción']]
  #    print("\nMatriz de Confusión:")
  #    print(confusion_matrix(y_true, y_pred))
  #    print("\nInforme de Clasificación:")
  #    print(classification_report(y_true, y_pred))

  # ROC-AUC para entrenamiento y prueba
  if not X_train_full.empty and not X_test.empty:
      X_train_scaled = StandardScaler().fit_transform(X_train_full)
      X_test_scaled = StandardScaler().fit_transform(X_test)
      best_model.fit(X_train_scaled, y_train_full)
      train_pred_proba = best_model.predict_proba(X_train_scaled)[:, 1]
      test_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]
      roc_auc_train = roc_auc_score(y_train_full, train_pred_proba)
      roc_auc_test = roc_auc_score(y_test, test_pred_proba).round(6)
      # Verificar tamaños de las muestras
      print(f"Tamaño de y_train_full: {y_train_full.size}")
      print(f"Tamaño de y_test: {y_test.size}")
      print("Distribución de clases en y_train_full:")
      print(y_train_full.value_counts())
      ratio_1_train=(y_train_full.value_counts()[1]/y_train_full.size).round(4)
      print(f"% clase 1 train: {ratio_1_train} ")

      print("Distribución de clases en y_test:")
      print(y_test.value_counts())
      ratio_1_test=(y_test.value_counts()[1]/y_test.size).round(4)
      print(f"% clase 1 test: {ratio_1_test} ")

      print(f"\nROC-AUC en el conjunto de entrenamiento: {roc_auc_train:.4f}")
      print(roc_auc_train)
      print(f"ROC-AUC en el conjunto de prueba: {roc_auc_test:.4f}")
      y_pred_test = (best_model.predict_proba(X_test_scaled)[:, 1] >= 0.3).astype(int)
      print("\nMatriz de Confusión (prueba):")
      print(confusion_matrix(y_test, y_pred_test))
      print("\nInforme de Clasificación (prueba):")
      print(classification_report(y_test, y_pred_test))
  else:
      print("\nAdvertencia: Conjunto de prueba o entrenamiento insuficiente. No se calculó ROC-AUC.")

  print("Mejores hiperparámetros:", random_search.best_params_)
  print("Mejor umbral:", best_threshold)


  # Predicción para el día siguiente
  last_features = df[features].iloc[-1:]
  scaler = StandardScaler()
  last_features_scaled = scaler.fit_transform(last_features)
  future_pred_prob = best_model.predict_proba(last_features_scaled)[0][1].round(4)
  future_pred = 1 if future_pred_prob >= best_threshold else 0

  returns = df['Return'].dropna()
  mean_return = returns.mean()
  std_return = returns.std()
  last_close = df['Close'].iloc[-1]
  expected_price = last_close * np.exp(mean_return + 0.5 * std_return**2)
  price_prob = future_pred_prob if future_pred == 1 else 1 - future_pred_prob

  action = 'BUY' if future_pred == 1 else 'SELL'
  direction = 1 if future_pred == 1 else -1

  resultsp.append({
              'Papel': papel,
              'Fecha Predicción': next_day,
              'Fecha Datos': df.index[-1],
              'Predicción': 'Alcista' if future_pred == 1 else 'Bajista',
              'Resultado Real': "Veremos",
              'Precio actual': last_close,
              'Precio Cierre': "futuro",
              'Correcta': "verificar",
              'Probabilidad Alcista': future_pred_prob,
              'Umbral': best_threshold,
              'ROC-AUC prueba': roc_auc_test ,
              'clase 1 en train': str(ratio_1_train),
              'clase 1 en test': str(ratio_1_test),
              'Mejores hiperparámetros': str(random_search.best_params_),
              'Matrix prueba': str(classification_report(y_test, y_pred_test))
              #si la pasas a tolist() perdes los titulos
              #'Matrix prueba': str((classification_report(y_test, y_pred_test)).tolist())

          })
  # Crear tabla de resultados
  resultsp_df = pd.DataFrame(resultsp)
  resultsp_df.set_index('Fecha Predicción', inplace=True)

  # Mostrar resultados
  pd.set_option('display.max_columns', None)
  print("\nPrediccion para el proximo dia (hasta", next_day.strftime('%Y-%m-%d'), "):")
  print("Nota: 'Fecha Predicción' es la fecha predicha; 'Fecha Datos' es la fecha de los datos usados.")
  print(resultsp_df)

  print(f"\nPredicción para {next_day.strftime('%Y-%m-%d')} (basada en datos hasta {df.index[-1].strftime('%Y-%m-%d')}):")
  print(f"Papel: {symbol}")
  print(f"Precio actual: [{last_close:.4f}]")
  print(f"Precio esperado para el siguiente día (distribución log-normal): [{expected_price:.4f}]")
  print(f"Probabilidad de que el precio predicho sea correcto: [{price_prob:.4f}]")
  print(f"Corte: {df.index[-1]}")
  print(f"\nPredicción para {next_day.strftime('%Y-%m-%d')}: {'Alcista' if future_pred == 1 else 'Bajista'} (Probabilidad Alcista: {future_pred_prob:.2%})")
  print(f"Pronóstico de dirección del activo (1: subida, -1: bajada): {direction}")
  print(f"Acción sugerida por la estrategia de trading: {action}")

  print(f"\n {symbol}, {next_day.strftime('%d-%m-%y')}, {df.index[-1].strftime('%d-%m-%y')}, {action}, , ,{future_pred_prob:.4}")


# Guardar y descargar el CSV
results_df.to_csv(f"backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv", sep=";")
files.download(f"backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv")
print(f"\nArchivo 'backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv' generado y descargado.")
import time
time.sleep(6)

resultsp_df.to_csv(f"Predic_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv", sep=";")
files.download(f"Predic_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv")
print(f"\nArchivo 'Predic_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv' generado y descargado.")

[*********************100%***********************]  1 of 1 completed


 Datos descargados en 1 vueltas: ALUA.BA , seguimos

 Datos descargados en 2 vueltas: ALUA.BA , seguimos

 Datos descargados en 3 vueltas: ALUA.BA , seguimos

 Datos descargados en 4 vueltas: ALUA.BA , seguimos

 Datos descargados en 5 vueltas: ALUA.BA , seguimos

 Datos descargados en 6 vueltas: ALUA.BA , seguimos

 Datos descargados en 7 vueltas: ALUA.BA , seguimos

 Datos descargados en 8 vueltas: ALUA.BA , seguimos

 Datos descargados en 9 vueltas: ALUA.BA , seguimos

 Datos descargados en 10 vueltas: ALUA.BA , seguimos

 Datos descargados en 11 vueltas: ALUA.BA , seguimos

 Datos descargados en 12 vueltas: ALUA.BA , seguimos

 Datos descargados en 13 vueltas: ALUA.BA , seguimos

 Datos descargados en 14 vueltas: ALUA.BA , seguimos

 Datos descargados en 15 vueltas: ALUA.BA , seguimos

 Datos descargados en 16 vueltas: ALUA.BA , seguimos

 Datos descargados en 17 vueltas: ALUA.BA , seguimos

 Datos descargados en 18 vueltas: ALUA.BA , seguimos

 Datos descargados en 19 vueltas: AL

Mejores hiperparámetros: {'subsample': 0.6, 'n_estimators': 900, 'max_depth': 5, 'learning_rate': 0.2, 'gamma': 0.1, 'colsample_bytree': 1.0}
Mejor umbral: 0.28
Escalando datos hasta 2025-07-18 00:00:00 para predecir 2025-07-21 00:00:00
Escalando datos hasta 2025-07-21 00:00:00 para predecir 2025-07-22 00:00:00

Resultados del backtesting (hasta 2025-07-23 ):
Nota: 'Fecha Predicción' es la fecha predicha; 'Fecha Datos' es la fecha de los datos usados.
                    Papel Fecha Datos Predicción Resultado Real  \
Fecha Predicción                                                  
2025-07-21        ALUA.BA  2025-07-18    Bajista        Bajista   
2025-07-22        ALUA.BA  2025-07-21    Bajista        Bajista   

                  Precio Cierre  Probabilidad Alcista Correcta  
Fecha Predicción                                                
2025-07-21                708.0                0.0002       Sí  
2025-07-22                689.0                0.0001       Sí  
Tamaño de y_tra

[*********************100%***********************]  1 of 1 completed



 Datos descargados en 1 vueltas: BBAR.BA , seguimos

 Datos descargados en 2 vueltas: BBAR.BA , seguimos

 Datos descargados en 3 vueltas: BBAR.BA , seguimos

 Datos descargados en 4 vueltas: BBAR.BA , seguimos

 Datos descargados en 5 vueltas: BBAR.BA , seguimos

 Datos descargados en 6 vueltas: BBAR.BA , seguimos

 Datos descargados en 7 vueltas: BBAR.BA , seguimos

 Datos descargados en 8 vueltas: BBAR.BA , seguimos

 Datos descargados en 9 vueltas: BBAR.BA , seguimos

 Datos descargados en 10 vueltas: BBAR.BA , seguimos

 Datos descargados en 11 vueltas: BBAR.BA , seguimos

 Datos descargados en 12 vueltas: BBAR.BA , seguimos

 Datos descargados en 13 vueltas: BBAR.BA , seguimos

 Datos descargados en 14 vueltas: BBAR.BA , seguimos

 Datos descargados en 15 vueltas: BBAR.BA , seguimos

 Datos descargados en 16 vueltas: BBAR.BA , seguimos

 Datos descargados en 17 vueltas: BBAR.BA , seguimos

 Datos descargados en 18 vueltas: BBAR.BA , seguimos

 Datos descargados en 19 vueltas: BB

[*********************100%***********************]  1 of 1 completed



 Datos descargados en 1 vueltas: BMA.BA , seguimos

 Datos descargados en 2 vueltas: BMA.BA , seguimos

 Datos descargados en 3 vueltas: BMA.BA , seguimos

 Datos descargados en 4 vueltas: BMA.BA , seguimos

 Datos descargados en 5 vueltas: BMA.BA , seguimos

 Datos descargados en 6 vueltas: BMA.BA , seguimos

 Datos descargados en 7 vueltas: BMA.BA , seguimos

 Datos descargados en 8 vueltas: BMA.BA , seguimos

 Datos descargados en 9 vueltas: BMA.BA , seguimos

 Datos descargados en 10 vueltas: BMA.BA , seguimos

 Datos descargados en 11 vueltas: BMA.BA , seguimos

 Datos descargados en 12 vueltas: BMA.BA , seguimos

 Datos descargados en 13 vueltas: BMA.BA , seguimos

 Datos descargados en 14 vueltas: BMA.BA , seguimos

 Datos descargados en 15 vueltas: BMA.BA , seguimos

 Datos descargados en 16 vueltas: BMA.BA , seguimos

 Datos descargados en 17 vueltas: BMA.BA , seguimos

 Datos descargados en 18 vueltas: BMA.BA , seguimos

 Datos descargados en 19 vueltas: BMA.BA , seguimos
Mu

KeyboardInterrupt: 

COMIENZA EL UMBRAL

In [ ]:
import pandas as pd
import yfinance as yf
import datetime as dt

# Descargar datos (si no lo hiciste aún)
end_date = dt.datetime(2025, 7, 20)
df = yf.download("COME.BA", start=dt.datetime(2001, 1, 1), end=end_date)

# Definir umbral (por ejemplo, 2%)
umbral = 0.02

# Calcular la diferencia relativa y etiquetar
df['Label'] = ((df['High'] - df['Open']) / df['Open'] > umbral).astype(int) * 2 - 1

# Contar etiquetas y calcular porcentajes
label_counts = df['Label'].value_counts()
total_dias = len(df)
percentages = (label_counts / total_dias) * 100

# Imprimir resultados
print("Conteo de etiquetas:")
print(label_counts)
print("\nPorcentajes de cada clase (%):")
print(percentages.round(2))

# Opcional: Ver los primeros días etiquetados
print("\nPrimeros días etiquetados:")
print(df[['Open', 'High', 'Label']].tail(20))
print("mambral")
umbral = df['High'].sub(df['Open']).div(df['Open']).quantile(0.75)
print(umbral)
df['Label'] = ((df['High'] - df['Open']) / df['Open'] > umbral).astype(int) * 2 - 1
print(df['Label'].value_counts(normalize=True) * 100)

# Contar etiquetas y calcular porcentajes
label_counts = df['Label'].value_counts()
total_dias = len(df)
percentages = (label_counts / total_dias) * 100

# Imprimir resultados
print("Conteo de etiquetas:")
print(label_counts)
print("\nPorcentajes de cada clase (%):")
print(percentages.round(2))

In [4]:
import pandas as pd
import yfinance as yf
import datetime as dt

# Descargar datos
end_date = dt.datetime(2025, 7, 30)
df = yf.download("GGAL", start=dt.datetime(2001, 1, 1), end=end_date)

# Aplanar el MultiIndex a columnas simples
df.columns = df.columns.map(lambda x: x[0])

# Limpiar datos
df = df.dropna(subset=['Open', 'High'])

# Calcular umbral dinámico
#differences = (df['High'] - df['Open']) / df['Open']
#umbral = differences.quantile(0.6).item()
#print(f"Umbral calculado: {umbral:.4f}")
umbral = 0.019
lapso = 2

# Calcular etiqueta sin desfase (para verificar)
df['Label_raw'] = ((df['High'] - df['Open']) / df['Open'] > umbral).astype(int) * 2 - 1

# Desplazar la etiqueta un día hacia atrás (target del día "lapso")
df['Label'] = df['Label_raw'].shift(-lapso)

# Eliminar la última fila (no tiene etiqueta para predecir)
df = df.dropna(subset=['Label'])
print(df.tail(33))
# Contar etiquetas y calcular porcentajes
label_counts = df['Label'].value_counts()
total_dias = len(df)
percentages = (label_counts / total_dias) * 100

print("\nConteo de etiquetas (desfasadas):")
print(label_counts)
print("\nPorcentajes de cada clase (%):")
print(percentages.round(2))

# Opcional: Ver los primeros días etiquetados
print("\nPrimeros días etiquetados (features del día anterior, label del día siguiente):")
print(df[['Open', 'High', 'Label']].tail(22))

/tmp/ipython-input-4-1161638459.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("GGAL", start=dt.datetime(2001, 1, 1), end=end_date)
[*********************100%***********************]  1 of 1 completed

                Close       High        Low       Open   Volume  Label_raw  \
Date                                                                         
2025-06-09  53.860058  55.365585  53.710500  55.206059   973800         -1   
2025-06-10  57.160259  57.748509  54.089377  54.358578  1459600          1   
2025-06-11  56.033604  56.113365  52.942780  54.837156  7670800          1   
2025-06-12  55.973782  57.100437  54.587898  55.235975  2270100          1   
2025-06-13  54.508133  55.834197  53.919883  55.385529  1772100         -1   
2025-06-16  54.338634  55.425407  54.089375  54.916919  1057500         -1   
2025-06-17  54.189079  54.807246  52.663611  53.620767  1329000          1   
2025-06-18  53.989674  55.156208  53.531033  54.129258   979700         -1   
2025-06-20  51.846039  53.919882  51.776246  53.919882  1124700         -1   
2025-06-23  50.220863  52.165091  49.911784  51.327577  1593000         -1   
2025-06-24  52.524025  53.291746  50.848999  50.848999  1284100 

hasta aca llegaste

**apruebas**

In [8]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from google.colab import files  # Para descargar el CSV en Colab
from scipy import stats


# Funciones de features
#def add_lagged_price_features(df):
#    for lag in range(1, 6):
#        df[f'close_lag_{lag}'] = df['Close'].shift(lag)
#    return df

def add_lagged_price_features(df, etiqueta="close_lag", dato="Close"):
    for lag in range(1, 6):
        df[f'{etiqueta}_{lag}'] = df[dato].shift(lag)
    return df

def calculate_RSI(series, period=7):
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_ROC(series, period=5):
    return ((series - series.shift(period)) / series.shift(period)) * 100

def calculate_PPO(series, fast_period=5, slow_period=9, signal_period=5):
    ema_fast = series.ewm(span=fast_period, adjust=False).mean()
    ema_slow = series.ewm(span=slow_period, adjust=False).mean()
    ppo = (ema_fast - ema_slow) / ema_slow * 100
    signal_line = ppo.ewm(span=signal_period, adjust=False).mean()
    histogram = ppo - signal_line
    return ppo, signal_line, histogram

def calculate_EWO(series, fast_period=5, slow_period=35, signal_period=5):
    ema_fast = series.ewm(span=fast_period, adjust=False).mean()
    ema_slow = series.ewm(span=slow_period, adjust=False).mean()
    ewo = (ema_fast - ema_slow) / ema_slow * 100
    signal_line = ewo.ewm(span=signal_period, adjust=False).mean()
    histogram = ewo - signal_line
    return ewo, signal_line, histogram

def calculate_volatility(series, window=20):
    return series.rolling(window).std().round(6)

def calculate_sma5(series, period=5):
    return series.rolling(window=period).mean().round(4)

def calculate_sma13(series, period=13):
    return series.rolling(window=period).mean().round(4)

def calculate_sma26(series, period=26):
    return series.rolling(window=period).mean().round(4)

def calculate_sma50(series, period=50):
    return series.rolling(window=period).mean().round(4)

def calculate_sma200(series, period=200):
    return series.rolling(window=period).mean().round(4)


#def calculate_earnings_season(df):
#    df['Is_Earnings_Season'] = df.index.month.isin([1, 4, 7, 10])
#    return df

#def calculate_christmas_rally(df):
#    df['Is_Christmas_Rally'] = df.index.month.isin([11, 12])
#    return df

def create_features(df):
    df = add_lagged_price_features(df, "close_lag", "Close")
    df = add_lagged_price_features(df, "open_lag", "Open")
    df = add_lagged_price_features(df, "high_lag", "High")
    df['Pct_change'] = df['Close'].pct_change()
    for lag in range(1, 6):
        df[f'lag_change{lag}'] = df['Pct_change'].shift(lag)
    df['RSI'] = calculate_RSI(df['Close'])
    df['ROC'] = calculate_ROC(df['Close'])
    df['PPO'], df['PPO_Signal'], df['PPO_Histogram'] = calculate_PPO(df['Close'])
    df['EWO'], df['EWO_Signal'], df['EWO_Histogram'] = calculate_EWO(df['Close'])
    df['SMA5'] = calculate_sma5(df['Close'])
    df['SMA13'] = calculate_sma13(df['Close'])
    df['SMA26'] = calculate_sma26(df['Close'])
    df['SMA50'] = calculate_sma50(df['Close'])
    df['SMA200'] = calculate_sma200(df['Close'])
    df['Volatility'] = calculate_volatility(df['Close'])
    # Calcular etiqueta sin desfase (para verificar)
    df['Label_raw'] = ((df['High'] - df['Open']) / df['Open'] > umbral).astype(int)
    # Desplazar la etiqueta un día hacia atrás (target del día "lapso")
    df['Label'] = df['Label_raw'].shift(-lapso)

    # Eliminar la última fila (no tiene etiqueta para predecir)
    #df = df.dropna(subset=['Label'])

    df['Return'] = np.log(df['Close'] / df['Close'].shift())
    #df = calculate_earnings_season(df)
    #df = calculate_christmas_rally(df)
    df.dropna(inplace=True)
    return df

# Definir fecha de corte manualmente (cambiar diariamente)
end_date = dt.datetime(2025, 8, 7)  # Ejemplo: cambiar a 2025-07-18 mañana

tk =[ "ALUA.BA", "BBAR.BA", "BMA.BA", "COME.BA", "CRES.BA", "EDN.BA", "GGAL.BA", "IRSA.BA", "LOMA.BA", "METR.BA", "PAMP.BA", "SUPV.BA", "TECO2.BA", "TGNO4.BA", "TGSU2.BA", "TRAN.BA", "TXAR.BA", "VALO.BA", "YPFD.BA"]

# pinchas = "BYMA.BA" start_date dt.datetime(2021, 1, 1),  "CEPU.BA" start_date  dt.datetime(2018, 1, 1)

results = [] # el del test, para que no lo reinicie
resultsp = [] # las predicciones, para que no lo reinicie

umbral = 0.019
lapso = 1

for papel in tk:

  symbol=papel
  #symbol="COME.BA"
  # Fechas dinámicas
  start_date = dt.datetime(2001, 1, 1)  # Inicio fijo
  train_end = end_date - pd.Timedelta(days=780)  # 6 meses antes de end_date (ajustable)
  next_day = end_date + pd.Timedelta(days=1)  # Predicción para el día siguiente
  backtest_start = end_date - pd.Timedelta(days=2)  # Inicio del backtesting 6 meses antes de end_date (ajustable)

  # Descargar datos
  df = yf.download(symbol, start=start_date, end=end_date, auto_adjust=False)

  # Verificar datos

  for intentos in range(1, 20):
    if df.empty:
      print("No se pudieron descargar datos. Verifica el símbolo, las fechas o la conexión.")
      print(f"\n Reintentando {symbol}: {intentos} de 20")
      df = yf.download(symbol, start=start_date, end=end_date, auto_adjust=False)
      intentos +=1
      time.sleep(8)
    else:
      print(f"\n Datos descargados en {intentos} vueltas: {symbol} , seguimos")


  # Aplanar MultiIndex si existe
  if isinstance(df.columns, pd.MultiIndex):
      print("MultiIndex detectado en columnas. Aplanando...")
      df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
      print("Columnas asignadas después de aplanamiento:", df.columns.tolist())
      #print("Últimas filas antes de corrección:", df.tail())

      # Corregir el orden de las columnas según tu mapeo
      columns = df.columns.tolist()
      df = df[[columns[4], columns[2], columns[3], columns[0], columns[5], columns[1]]]  # Reordenar
      df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']  # Asignar nombres correctos
      #print("Últimas filas después de corrección:", df.tail())

  if isinstance(df.index, pd.MultiIndex):
      print("MultiIndex detectado en índice. Seleccionando ticker...")
      df = df.xs(symbol, level='Ticker', axis=0)
  df.index = pd.to_datetime(df.index)  # Asegurar índice datetime
  if not df.index.is_unique:
      print("Advertencia: Índice con fechas duplicadas. Eliminando duplicados...")
      df = df[~df.index.duplicated(keep='first')]

  # Verificar columnas
  print("Columnas del DataFrame después de descargar y corregir:")
  #print(df.columns)

  df['Open']= df['Open'].round(2)
  df['High']= df['High'].round(2)
  df['Low']= df['Low'].round(2)
  df['Close']= df['Close'].round(2)
  df['Adj Close']= df['Adj Close'].round(2)

  print("Últimas filas del DataFrame antes de crear features:")
  print(df.tail())







  # Crear features
  df = create_features(df)

  # Verificar datos después de crear features
  print("\nÚltimas filas del DataFrame después de crear features:")
  print(df.tail())
  print(df.columns)

  print("Distribucion de etiquetas")
  print(df["Label"].value_counts(normalize=True))
  correlation = df[features + ["Label"]].corr()["Label"].sort_values(ascending=False)
  print("Correlacion con label")
  print(correlation)


  # Seleccionar features
  features = ['RSI', 'ROC', 'PPO', 'PPO_Signal', 'PPO_Histogram', 'EWO', 'EWO_Signal', 'EWO_Histogram', 'Volatility', 'SMA5', 'SMA13', 'SMA26', 'SMA50', 'SMA200' ] + [f'lag_change{i}' for i in range(1, 6)] + \
            [f'close_lag_{i}' for i in range(1, 6)] + [f'open_lag_{i}' for i in range(1, 6)]+ [f'high_lag_{i}' for i in range(1, 6)]
  X = df[features]
  y = df['Label']

  # Dividir datos en entrenamiento y prueba
  X_train_full = X[df.index <= train_end]
  y_train_full = y[df.index <= train_end]
  X_test = X[(df.index > train_end) & (df.index <= end_date)]  # Hasta end_date
  y_test = y[(df.index > train_end) & (df.index <= end_date)]

  # Optimizar hiperparámetros con RandomizedSearchCV
  print("Optimizar hiperparámetros con RandomizedSearchCV")
  param_dist = {
      'learning_rate': [0.01, 0.05, 0.1, 0.2],
      'max_depth': [3, 5, 7, 9],
      'n_estimators': [100, 500, 900],
      'subsample': [0.6, 0.8, 1.0],
      'colsample_bytree': [0.6, 0.8, 1.0],
      'gamma': [0, 0.1, 0.2]
  }
  xgb = XGBClassifier(objective='binary:logistic', random_state=42)
  random_search = RandomizedSearchCV(xgb, param_distributions=param_dist, n_iter=20, cv=5, scoring='f1', n_jobs=-1, random_state=42)
  random_search.fit(X_train_full, y_train_full)
  print("Mejores hiperparámetros:", random_search.best_params_)

  # Usar el mejor modelo
  best_model = random_search.best_estimator_

  # Optimizar el umbral
  y_train_prob = best_model.predict_proba(X_train_full)[:, 1]
  thresholds = np.arange(0.28, 0.88, 0.01) # de 0.28 a 0.88  con pasos de 0.01????
  best_threshold = 0.5
  best_f1 = 0
  for threshold in thresholds:
      y_pred_threshold = (y_train_prob >= threshold).astype(int)
      f1 = f1_score(y_train_full, y_pred_threshold)
      if f1 > best_f1:
          best_f1 = f1
          best_threshold = threshold
  best_threshold.round(2)
  print("Mejor umbral:", best_threshold)



  # Backtesting con escalado dinámico para evitar data leak
  dates = df[(df.index >= backtest_start) & (df.index <= end_date)].index
  #results = []

  for test_date in dates:
      train_data = df[df.index < test_date].copy()
      if train_data.empty or train_data['Label'].isna().all():
          continue
      train_data = train_data.dropna()

      X_train_loop = train_data[features]
      y_train_loop = train_data['Label']

      scaler = StandardScaler()  # Reinicio dinámico del escalador
      X_train_scaled = scaler.fit_transform(X_train_loop)
      print(f"Escalando datos hasta {train_data.index[-1]} para predecir {test_date}")

      best_model.fit(X_train_scaled, y_train_loop)

      if test_date in df.index:
          test_row = df.loc[[test_date]][features]
          if test_row.empty:
              continue
          test_row_scaled = scaler.transform(test_row)

          prediction_prob = best_model.predict_proba(test_row_scaled)[0][1].round(4)
          prediction = 1 if prediction_prob >= best_threshold else 0

          real_direction = df.loc[test_date, 'Label'] if pd.notna(df.loc[test_date, 'Label']) else None
          open_price = df.loc[test_date, 'Open']
          max_price = df.loc[test_date, 'High']
          ganance = ((max_price-open_price)/open_price).round(4)

          is_correct = int(prediction == real_direction) if real_direction is not None else None
          data_date = df.index[df.index < test_date][-1] if not df[df.index < test_date].empty else None

          results.append({
              'Papel': papel,
              'Fecha Predicción': test_date,
              'Fecha Datos': data_date,
              'Predicción': 'Alcista' if prediction == 1 else 'Bajista',
              'Resultado Real': 'Alcista' if real_direction == 1 else 'Bajista' if real_direction == 0 else None,
              'Precio Apertura': open_price,
              'Precio Máximo': max_price,
              'Mov_maximo': ganance,
              'Probabilidad Alcista': prediction_prob,
              'Correcta': 'Sí' if is_correct == 1 else 'No' if is_correct == 0 else None
          })

  # Crear tabla de resultados
  results_df = pd.DataFrame(results)
  print("ver si falta la fila")
  print(results_df)
  results_df.set_index('Fecha Predicción', inplace=True)

  # Mostrar resultados
  pd.set_option('display.max_columns', None)
  print("\nResultados del backtesting (hasta", end_date.strftime('%Y-%m-%d'), "):")
  print("Nota: 'Fecha Predicción' es la fecha predicha; 'Fecha Datos' es la fecha de los datos usados.")
  print(results_df)

  # Guardar y descargar el CSV
  #results_df.to_csv(f"backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv", sep=";")
  #files.download(f"backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv")
  #print(f"\nArchivo 'backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv' generado y descargado.")

  # Métricas del backtesting
  #if results_df['Correcta'].notna().sum() > 0:
  #    accuracy = (results_df['Correcta'] == 'Sí').sum() / results_df['Correcta'].notna().sum()
  #    print(f"\nAccuracy del backtesting: {accuracy:.2%}")

  #    valid_results = results_df[results_df['Correcta'].notna()]
  #    y_true = [1 if r == 'Alcista' else 0 for r in valid_results['Resultado Real']]
  #    y_pred = [1 if p == 'Alcista' else 0 for p in valid_results['Predicción']]
  #    print("\nMatriz de Confusión:")
  #    print(confusion_matrix(y_true, y_pred))
  #    print("\nInforme de Clasificación:")
  #    print(classification_report(y_true, y_pred))

  # ROC-AUC para entrenamiento y prueba
  if not X_train_full.empty and not X_test.empty:
      X_train_scaled = StandardScaler().fit_transform(X_train_full)
      X_test_scaled = StandardScaler().fit_transform(X_test)
      best_model.fit(X_train_scaled, y_train_full)
      train_pred_proba = best_model.predict_proba(X_train_scaled)[:, 1]
      test_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]
      roc_auc_train = roc_auc_score(y_train_full, train_pred_proba)
      roc_auc_test = roc_auc_score(y_test, test_pred_proba).round(6)
      # Verificar tamaños de las muestras
      print(f"Tamaño de y_train_full: {y_train_full.size}")
      print(f"Tamaño de y_test: {y_test.size}")
      print("Distribución de clases en y_train_full:")
      print(y_train_full.value_counts())
      ratio_1_train=(y_train_full.value_counts()[1]/y_train_full.size).round(4)
      print(f"% clase 1 train: {ratio_1_train} ")

      print("Distribución de clases en y_test:")
      print(y_test.value_counts())
      ratio_1_test=(y_test.value_counts()[1]/y_test.size).round(4)
      print(f"% clase 1 test: {ratio_1_test} ")

      print(f"\nROC-AUC en el conjunto de entrenamiento: {roc_auc_train:.4f}")
      print(roc_auc_train)
      print(f"ROC-AUC en el conjunto de prueba: {roc_auc_test:.4f}")
      y_pred_test = (best_model.predict_proba(X_test_scaled)[:, 1] >= 0.3).astype(int)
      print("\nMatriz de Confusión (prueba):")
      print(confusion_matrix(y_test, y_pred_test))
      print("\nInforme de Clasificación (prueba):")
      print(classification_report(y_test, y_pred_test))
  else:
      print("\nAdvertencia: Conjunto de prueba o entrenamiento insuficiente. No se calculó ROC-AUC.")

  print("Mejores hiperparámetros:", random_search.best_params_)
  print("Mejor umbral:", best_threshold)


  # Predicción para el día siguiente
  last_features = df[features].iloc[-1:]
  scaler = StandardScaler()
  last_features_scaled = scaler.fit_transform(last_features)
  future_pred_prob = best_model.predict_proba(last_features_scaled)[0][1].round(4)
  future_pred = 1 if future_pred_prob >= best_threshold else 0

  returns = df['Return'].dropna()
  mean_return = returns.mean()
  std_return = returns.std()
  last_close = df['Close'].iloc[-1]
  last_open = df['Open'].iloc[-1]
  last_max = df['High'].iloc[-1]

  expected_price = last_close * np.exp(mean_return + 0.5 * std_return**2)
  price_prob = future_pred_prob if future_pred == 1 else 1 - future_pred_prob

  action = 'BUY' if future_pred == 1 else 'SELL'
  direction = 1 if future_pred == 1 else -1

  resultsp.append({
              'Papel': papel,
              'Fecha Predicción': next_day,
              'Fecha Datos': df.index[-1],
              'Predicción': 'Alcista' if future_pred == 1 else 'Bajista',
              'Resultado Real': "Veremos",
              'Precio actual': last_close,
              'Precio apertura': last_open,
              'Precio máximo': last_max,
              'Precio Cierre': "futuro",
              'Correcta': "verificar",
              'Probabilidad Alcista': future_pred_prob,
              'Umbral': best_threshold.round(2),
              'ROC-AUC prueba': roc_auc_test ,
              'clase 1 en train': str(ratio_1_train),
              'clase 1 en test': str(ratio_1_test),
              'Mejores hiperparámetros': str(random_search.best_params_),
              'Matrix prueba': str(classification_report(y_test, y_pred_test))
              #si la pasas a tolist() perdes los titulos
              #'Matrix prueba': str((classification_report(y_test, y_pred_test)).tolist())

          })
  # Crear tabla de resultados
  resultsp_df = pd.DataFrame(resultsp)
  print(resultsp_df)
  resultsp_df.set_index('Fecha Predicción', inplace=True)

  # Mostrar resultados
  pd.set_option('display.max_columns', None)
  print("\nPrediccion para el proximo dia (hasta", next_day.strftime('%Y-%m-%d'), "):")
  print("Nota: 'Fecha Predicción' es la fecha predicha; 'Fecha Datos' es la fecha de los datos usados.")
  print(resultsp_df)

  print(f"\nPredicción para {next_day.strftime('%Y-%m-%d')} (basada en datos hasta {df.index[-1].strftime('%Y-%m-%d')}):")
  print(f"Papel: {symbol}")
  print(f"Precio actual: [{last_close:.4f}]")
  print(f"Precio esperado para el siguiente día (distribución log-normal): [{expected_price:.4f}]")
  print(f"Probabilidad de que el precio predicho sea correcto: [{price_prob:.4f}]")
  print(f"Corte: {df.index[-1]}")
  print(f"\nPredicción para {next_day.strftime('%Y-%m-%d')}: {'Alcista' if future_pred == 1 else 'Bajista'} (Probabilidad Alcista: {future_pred_prob:.2%})")
  print(f"Pronóstico de dirección del activo (1: subida, -1: bajada): {direction}")
  print(f"Acción sugerida por la estrategia de trading: {action}")

  print(f"\n {symbol}, {next_day.strftime('%d-%m-%y')}, {df.index[-1].strftime('%d-%m-%y')}, {action}, , ,{future_pred_prob:.4}")


# Guardar y descargar el CSV
results_df.to_csv(f"backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv", sep=";")
files.download(f"backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv")
print(f"\nArchivo 'backtesting_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv' generado y descargado.")
import time
time.sleep(6)

resultsp_df.to_csv(f"Predic_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv", sep=";")
files.download(f"Predic_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv")
print(f"\nArchivo 'Predic_results_{symbol}_{end_date.strftime('%Y-%m-%d')}.csv' generado y descargado.")

[*********************100%***********************]  1 of 1 completed


 Datos descargados en 1 vueltas: ALUA.BA , seguimos

 Datos descargados en 2 vueltas: ALUA.BA , seguimos

 Datos descargados en 3 vueltas: ALUA.BA , seguimos

 Datos descargados en 4 vueltas: ALUA.BA , seguimos

 Datos descargados en 5 vueltas: ALUA.BA , seguimos

 Datos descargados en 6 vueltas: ALUA.BA , seguimos

 Datos descargados en 7 vueltas: ALUA.BA , seguimos

 Datos descargados en 8 vueltas: ALUA.BA , seguimos

 Datos descargados en 9 vueltas: ALUA.BA , seguimos

 Datos descargados en 10 vueltas: ALUA.BA , seguimos

 Datos descargados en 11 vueltas: ALUA.BA , seguimos

 Datos descargados en 12 vueltas: ALUA.BA , seguimos

 Datos descargados en 13 vueltas: ALUA.BA , seguimos

 Datos descargados en 14 vueltas: ALUA.BA , seguimos

 Datos descargados en 15 vueltas: ALUA.BA , seguimos

 Datos descargados en 16 vueltas: ALUA.BA , seguimos

 Datos descargados en 17 vueltas: ALUA.BA , seguimos

 Datos descargados en 18 vueltas: ALUA.BA , seguimos

 Datos descargados en 19 vueltas: AL

Mejores hiperparámetros: {'subsample': 0.6, 'n_estimators': 900, 'max_depth': 5, 'learning_rate': 0.2, 'gamma': 0.1, 'colsample_bytree': 1.0}
Mejor umbral: 0.28
Escalando datos hasta 2025-08-04 00:00:00 para predecir 2025-08-05 00:00:00
ver si falta la fila
     Papel Fecha Predicción Fecha Datos Predicción Resultado Real  \
0  ALUA.BA       2025-08-05  2025-08-04    Bajista        Alcista   

   Precio Apertura  Precio Máximo  Mov_maximo  Probabilidad Alcista Correcta  
0            748.0          753.0      0.0067                0.0038       No  

Resultados del backtesting (hasta 2025-08-07 ):
Nota: 'Fecha Predicción' es la fecha predicha; 'Fecha Datos' es la fecha de los datos usados.
                    Papel Fecha Datos Predicción Resultado Real  \
Fecha Predicción                                                  
2025-08-05        ALUA.BA  2025-08-04    Bajista        Alcista   

                  Precio Apertura  Precio Máximo  Mov_maximo  \
Fecha Predicción                    

[*********************100%***********************]  1 of 1 completed



 Datos descargados en 1 vueltas: BBAR.BA , seguimos

 Datos descargados en 2 vueltas: BBAR.BA , seguimos

 Datos descargados en 3 vueltas: BBAR.BA , seguimos

 Datos descargados en 4 vueltas: BBAR.BA , seguimos

 Datos descargados en 5 vueltas: BBAR.BA , seguimos

 Datos descargados en 6 vueltas: BBAR.BA , seguimos

 Datos descargados en 7 vueltas: BBAR.BA , seguimos

 Datos descargados en 8 vueltas: BBAR.BA , seguimos

 Datos descargados en 9 vueltas: BBAR.BA , seguimos

 Datos descargados en 10 vueltas: BBAR.BA , seguimos

 Datos descargados en 11 vueltas: BBAR.BA , seguimos

 Datos descargados en 12 vueltas: BBAR.BA , seguimos

 Datos descargados en 13 vueltas: BBAR.BA , seguimos

 Datos descargados en 14 vueltas: BBAR.BA , seguimos

 Datos descargados en 15 vueltas: BBAR.BA , seguimos

 Datos descargados en 16 vueltas: BBAR.BA , seguimos

 Datos descargados en 17 vueltas: BBAR.BA , seguimos

 Datos descargados en 18 vueltas: BBAR.BA , seguimos

 Datos descargados en 19 vueltas: BB

KeyboardInterrupt: 